In [1]:
import flopy
import os
import flopy.utils.binaryfile as bf
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt

In [2]:
#create simulation name and workspace
ws = 'data/ex01b'
name = 'ex01b'
exe_name = os.path.abspath('../bin/mf6')

# create simulation
sim = flopy.mf6.MFSimulation(sim_name=name, exe_name=exe_name, sim_ws=ws)

# create tdis package
tdis = flopy.mf6.ModflowTdis(sim)

# create iterative model solution
ims = flopy.mf6.ModflowIms(sim)

In [3]:
#Open and read raster files
#Raster paths
#Modell übernimmt die gleiche spatial resoulution wie rasterdata
demPath ="../Rst/DEM_200.tif"
crPath = "../Rst/CR.tif"

#Open files
demDs =gdal.Open(demPath)
crDs = gdal.Open(crPath)
geot = crDs.GetGeoTransform() #Xmin, deltax, ?, ymax, ?, delta y

geot

(613882.5068, 200.0, 0.0, 8375404.9524, 0.0, -200.0)

In [4]:
# Get data as arrays
demData = demDs.GetRasterBand(1).ReadAsArray()
crData = crDs.GetRasterBand(1).ReadAsArray()

# Get statistics
stats = demDs.GetRasterBand(1).GetStatistics(0,1) 
stats

[3396.0, 4873.0, 4243.0958193041, 287.90182492684]

In [5]:
#Elevation of layers
Layer1 = demData - 7.5 #demData ist Oberfläche
Layer1_2 = demData - 15
Layer2 = (demData - 3000)*0.8 + 3000
Layer3 = (demData - 3000)*0.5 + 3000
Layer4 = (demData>0)*3000

In [6]:
#Boundaries for Dis = Create discretization object, spatial/temporal discretization
ztop = demData
zbot = [Layer1, Layer1_2, Layer2, Layer3, Layer4]
nlay = 5
nrow = demDs.RasterYSize
ncol = demDs.RasterXSize
delr = geot[1]
delc = abs(geot[5])

In [8]:
demData

array([[4344., 4329., 4299., ..., 4484., 4538., 4611.],
       [4365., 4315., 4309., ..., 4490., 4528., 4562.],
       [4384., 4332., 4327., ..., 4572., 4546., 4546.],
       ...,
       [3865., 3913., 3926., ..., 4016., 4036., 4067.],
       [3842., 3894., 3941., ..., 3979., 4018., 4041.],
       [3859., 3877., 3935., ..., 3969., 3985., 4017.]], dtype=float32)

In [7]:
#create gwf model
gwf = flopy.mf6.ModflowGwf(sim, modelname=name)
#crate dis-package
dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay,nrow=nlay,ncol=ncol,delr=delr,delc=delc,top=ztop,botm=zbot)

MFDataException: An error occurred in data element "top" model "ex01b" package "dis". The error occurred while setting data in the "__init__" method.
Additional Information:
(1) Unable to set data "top" layer 0.  Data is not in a valid format.
(2) Error occurred while adding dataset "top" to block "griddata"
